In [4]:
import random
import numpy as np
import os
import torch as torch
from load_data import load_EOD_data
from evaluator import evaluate
from model import get_loss, StockMixer
import pickle


In [12]:
np.random.seed(123456789)
torch.random.manual_seed(12345678)
device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'

data_path = '../dataset'
market_name = 'NASDAQ'
relation_name = 'wikidata'
stock_num = 1026
lookback_length = 16
epochs = 100
valid_index = 756
test_index = 1008
fea_num = 5
market_num = 20
steps = 1
learning_rate = 0.001
alpha = 0.1
scale_factor = 3
activation = 'GELU'

In [13]:
dataset_path = '../dataset/' + market_name
if market_name == "SP500":
    data = np.load('../dataset/SP500/SP500.npy')
    data = data[:, 915:, :]
    price_data = data[:, :, -1]
    mask_data = np.ones((data.shape[0], data.shape[1]))
    eod_data = data
    gt_data = np.zeros((data.shape[0], data.shape[1]))
    for ticket in range(0, data.shape[0]):
        for row in range(1, data.shape[1]):
            gt_data[ticket][row] = (data[ticket][row][-1] - data[ticket][row - steps][-1]) / \
                                   data[ticket][row - steps][-1]
else:
    with open(os.path.join(dataset_path, "/media/isk/New Volume/Kuliah/Semester_6/Business Intelligence/Code/StockMixer/dataset/NASDAQ/eod_data.pkl"), "rb") as f:
        eod_data = pickle.load(f)
    with open(os.path.join(dataset_path, "/media/isk/New Volume/Kuliah/Semester_6/Business Intelligence/Code/StockMixer/dataset/NASDAQ/mask_data.pkl"), "rb") as f:
        mask_data = pickle.load(f)
    with open(os.path.join(dataset_path, "/media/isk/New Volume/Kuliah/Semester_6/Business Intelligence/Code/StockMixer/dataset/NASDAQ/gt_data.pkl"), "rb") as f:
        gt_data = pickle.load(f)
    with open(os.path.join(dataset_path, "/media/isk/New Volume/Kuliah/Semester_6/Business Intelligence/Code/StockMixer/dataset/NASDAQ/price_data.pkl"), "rb") as f:
        price_data = pickle.load(f)

In [14]:
trade_dates = mask_data.shape[1]
model = StockMixer(
    stocks=stock_num,
    time_steps=lookback_length,
    channels=fea_num,
    market=market_num,
    scale=scale_factor
).to(device)

In [15]:
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
best_valid_loss = np.inf
best_valid_perf = None
best_test_perf = None
batch_offsets = np.arange(start=0, stop=valid_index, dtype=int)

In [17]:
def validate(start_index, end_index):
    with torch.no_grad():
        cur_valid_pred = np.zeros([stock_num, end_index - start_index], dtype=float)
        cur_valid_gt = np.zeros([stock_num, end_index - start_index], dtype=float)
        cur_valid_mask = np.zeros([stock_num, end_index - start_index], dtype=float)
        loss = 0.
        reg_loss = 0.
        rank_loss = 0.
        for cur_offset in range(start_index - lookback_length - steps + 1, end_index - lookback_length - steps + 1):
            data_batch, mask_batch, price_batch, gt_batch = map(

                lambda x: torch.Tensor(x).to(device),
                get_batch(cur_offset)
            )
            prediction = model(data_batch)
            cur_loss, cur_reg_loss, cur_rank_loss, cur_rr = get_loss(prediction, gt_batch, price_batch, mask_batch,
                                                                     stock_num, alpha)
            loss += cur_loss.item()
            reg_loss += cur_reg_loss.item()
            rank_loss += cur_rank_loss.item()
            cur_valid_pred[:, cur_offset - (start_index - lookback_length - steps + 1)] = cur_rr[:, 0].cpu()
            cur_valid_gt[:, cur_offset - (start_index - lookback_length - steps + 1)] = gt_batch[:, 0].cpu()
            cur_valid_mask[:, cur_offset - (start_index - lookback_length - steps + 1)] = mask_batch[:, 0].cpu()
        loss = loss / (end_index - start_index)
        reg_loss = reg_loss / (end_index - start_index)
        rank_loss = rank_loss / (end_index - start_index)
        cur_valid_perf = evaluate(cur_valid_pred, cur_valid_gt, cur_valid_mask)
    return loss, reg_loss, rank_loss, cur_valid_perf


def get_batch(offset=None):
    if offset is None:
        offset = random.randrange(0, valid_index)
    seq_len = lookback_length
    mask_batch = mask_data[:, offset: offset + seq_len + steps]
    mask_batch = np.min(mask_batch, axis=1)
    return (
        eod_data[:, offset:offset + seq_len, :],
        np.expand_dims(mask_batch, axis=1),
        np.expand_dims(price_data[:, offset + seq_len - 1], axis=1),
        np.expand_dims(gt_data[:, offset + seq_len + steps - 1], axis=1))


In [18]:
for epoch in range(epochs):
    print("epoch{}##########################################################".format(epoch + 1))
    np.random.shuffle(batch_offsets)
    tra_loss = 0.0
    tra_reg_loss = 0.0
    tra_rank_loss = 0.0
    for j in range(valid_index - lookback_length - steps + 1):
        data_batch, mask_batch, price_batch, gt_batch = map(
            lambda x: torch.Tensor(x).to(device),
            get_batch(batch_offsets[j])
        )
        optimizer.zero_grad()
        prediction = model(data_batch)
        cur_loss, cur_reg_loss, cur_rank_loss, _ = get_loss(prediction, gt_batch, price_batch, mask_batch,
                                                            stock_num, alpha)
        cur_loss = cur_loss
        cur_loss.backward()
        optimizer.step()

        tra_loss += cur_loss.item()
        tra_reg_loss += cur_reg_loss.item()
        tra_rank_loss += cur_rank_loss.item()
    tra_loss = tra_loss / (valid_index - lookback_length - steps + 1)
    tra_reg_loss = tra_reg_loss / (valid_index - lookback_length - steps + 1)
    tra_rank_loss = tra_rank_loss / (valid_index - lookback_length - steps + 1)
    print('Train : loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(tra_loss, tra_reg_loss, tra_rank_loss))

    val_loss, val_reg_loss, val_rank_loss, val_perf = validate(valid_index, test_index)
    print('Valid : loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(val_loss, val_reg_loss, val_rank_loss))

    test_loss, test_reg_loss, test_rank_loss, test_perf = validate(test_index, trade_dates)
    print('Test: loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(test_loss, test_reg_loss, test_rank_loss))

    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        best_valid_perf = val_perf
        best_test_perf = test_perf

    print('Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(val_perf['mse'], val_perf['IC'],
                                                     val_perf['RIC'], val_perf['prec_10'], val_perf['sharpe5']))
    print('Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(test_perf['mse'], test_perf['IC'],
                                                                            test_perf['RIC'], test_perf['prec_10'], test_perf['sharpe5']), '\n\n')


epoch1##########################################################


KeyboardInterrupt: 

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.linear(x)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import pickle

# Seed initialization
np.random.seed(123456789)
torch.manual_seed(12345678)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset and parameters
data_path = '../dataset'
market_name = 'SP500'
relation_name = 'wikidata'
stock_num = 1026
lookback_length = 16
epochs = 100
valid_index = 756
test_index = 1008
fea_num = 5
market_num = 20
steps = 1
learning_rate = 0.001
alpha = 0.1
scale_factor = 3

dataset_path = os.path.join('../dataset', market_name)
if market_name == "SP500":
    data = np.load(os.path.join(dataset_path, 'SP500.npy'))
    data = data[:, 915:, :]
    price_data = data[:, :, -1]
    mask_data = np.ones((data.shape[0], data.shape[1]))
    eod_data = data
    gt_data = np.zeros((data.shape[0], data.shape[1]))
    for ticket in range(data.shape[0]):
        for row in range(1, data.shape[1]):
            gt_data[ticket][row] = (data[ticket][row][-1] - data[ticket][row - steps][-1]) / data[ticket][row - steps][-1]
else:
    with open(os.path.join(dataset_path, 'eod_data.pkl'), 'rb') as f:
        eod_data = pickle.load(f)
    with open(os.path.join(dataset_path, 'mask_data.pkl'), 'rb') as f:
        mask_data = pickle.load(f)
    with open(os.path.join(dataset_path, 'gt_data.pkl'), 'rb') as f:
        gt_data = pickle.load(f)
    with open(os.path.join(dataset_path, 'price_data.pkl'), 'rb') as f:
        price_data = pickle.load(f)

# Model definition
input_dim = lookback_length * fea_num
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

batch_offsets = np.arange(0, valid_index, dtype=int)

def prepare_input_data(offset, lookback_length, steps):
    seq_len = lookback_length
    mask_batch = mask_data[:, offset: offset + seq_len + steps]
    mask_batch = np.min(mask_batch, axis=1)
    return (
        eod_data[:, offset:offset + seq_len, :].reshape(-1, input_dim),
        np.expand_dims(mask_batch, axis=1),
        np.expand_dims(price_data[:, offset + seq_len - 1], axis=1),
        np.expand_dims(gt_data[:, offset + seq_len + steps - 1], axis=1)
    )

def get_batch(offset=None):
    if offset is None:
        offset = np.random.randint(0, valid_index)
    data, mask, price, gt = prepare_input_data(offset, lookback_length, steps)
    return torch.tensor(data, dtype=torch.float32).to(device), torch.tensor(mask, dtype=torch.float32).to(device), torch.tensor(price, dtype=torch.float32).to(device), torch.tensor(gt, dtype=torch.float32).to(device)

def validate(start_index, end_index):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for offset in range(start_index, end_index):
            data, mask, price, gt = get_batch(offset)
            prediction = model(data)
            loss = criterion(prediction * mask, gt * mask)
            total_loss += loss.item()
    return total_loss / (end_index - start_index)

# Training loop
for epoch in range(epochs):
    model.train()
    np.random.shuffle(batch_offsets)
    train_loss = 0.0
    for j in range(valid_index - lookback_length - steps):
        data, mask, price, gt = get_batch(batch_offsets[j])
        optimizer.zero_grad()
        prediction = model(data)
        loss = criterion(prediction * mask, gt * mask)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= (valid_index - lookback_length - steps)
    print(f'Epoch {epoch + 1}, Training Loss: {train_loss:.4f}')

    val_loss = validate(valid_index, test_index)
    print(f'Epoch {epoch + 1}, Validation Loss: {val_loss:.4f}')

    test_loss = validate(test_index, len(mask_data))
    print(f'Epoch {epoch + 1}, Test Loss: {test_loss:.4f}')


NameError: name 'LinearRegressionModel' is not defined

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import pickle

# Seed initialization
np.random.seed(123456789)
torch.manual_seed(12345678)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset and parameters
data_path = '../dataset'
market_name = 'NASDAQ'
relation_name = 'wikidata'
stock_num = 1026
lookback_length = 16
epochs = 100
valid_index = 756
test_index = 1008
fea_num = 5
market_num = 20
steps = 1
learning_rate = 0.001
alpha = 0.1
scale_factor = 3

dataset_path = os.path.join('../dataset', market_name)
if market_name == "SP500":
    data = np.load(os.path.join(dataset_path, 'SP500.npy'))
    data = data[:, 915:, :]
    price_data = data[:, :, -1]
    mask_data = np.ones((data.shape[0], data.shape[1]))
    eod_data = data
    gt_data = np.zeros((data.shape[0], data.shape[1]))
    for ticket in range(data.shape[0]):
        for row in range(1, data.shape[1]):
            gt_data[ticket][row] = (data[ticket][row][-1] - data[ticket][row - steps][-1]) / data[ticket][row - steps][-1]
else:
    with open(os.path.join(dataset_path, '/media/isk/New Volume/Kuliah/Semester_6/Business Intelligence/Code/StockMixer/dataset/NASDAQ/eod_data.pkl'), 'rb') as f:
        eod_data = pickle.load(f)
    with open(os.path.join(dataset_path, '/media/isk/New Volume/Kuliah/Semester_6/Business Intelligence/Code/StockMixer/dataset/NASDAQ/mask_data.pkl'), 'rb') as f:
        mask_data = pickle.load(f)
    with open(os.path.join(dataset_path, '/media/isk/New Volume/Kuliah/Semester_6/Business Intelligence/Code/StockMixer/dataset/NASDAQ/gt_data.pkl'), 'rb') as f:
        gt_data = pickle.load(f)
    with open(os.path.join(dataset_path, '/media/isk/New Volume/Kuliah/Semester_6/Business Intelligence/Code/StockMixer/dataset/NASDAQ/price_data.pkl'), 'rb') as f:
        price_data = pickle.load(f)

# Define the LinearRegressionModel (add this if you don't already have it)
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

# Model definition
input_dim = lookback_length * fea_num
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

batch_offsets = np.arange(0, valid_index, dtype=int)

def prepare_input_data(offset, lookback_length, steps):
    seq_len = lookback_length
    mask_batch = mask_data[:, offset: offset + seq_len + steps]
    mask_batch = np.min(mask_batch, axis=1)
    return (
        eod_data[:, offset:offset + seq_len, :].reshape(-1, input_dim),
        np.expand_dims(mask_batch, axis=1),
        np.expand_dims(price_data[:, offset + seq_len - 1], axis=1),
        np.expand_dims(gt_data[:, offset + seq_len + steps - 1], axis=1)
    )

def get_batch(offset=None):
    if offset is None:
        offset = np.random.randint(0, valid_index)
    data, mask, price, gt = prepare_input_data(offset, lookback_length, steps)
    return torch.tensor(data, dtype=torch.float32).to(device), torch.tensor(mask, dtype=torch.float32).to(device), torch.tensor(price, dtype=torch.float32).to(device), torch.tensor(gt, dtype=torch.float32).to(device)

def mean_squared_error(y_true, y_pred):
    return torch.mean((y_true - y_pred) ** 2)

def information_coefficient(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return torch.corrcoef(torch.stack([y_true, y_pred]))[0, 1]

def rank_information_coefficient(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return torch.corrcoef(torch.stack([torch.argsort(y_true), torch.argsort(y_pred)]))[0, 1]

def precision_at_k(y_true, y_pred, k=10):
    _, topk_indices = torch.topk(y_pred, k, dim=0)
    precision = torch.sum(y_true[topk_indices]) / k
    return precision

def sharpe_ratio(y_true, y_pred):
    excess_returns = y_true - y_pred
    return torch.mean(excess_returns) / torch.std(excess_returns)

def validate(start_index, end_index):
    model.eval()
    total_loss = 0
    total_ic = 0
    total_ric = 0
    total_prec_10 = 0
    total_sharpe5 = 0
    count = 0
    with torch.no_grad():
        for offset in range(start_index, end_index):
            data, mask, price, gt = get_batch(offset)
            prediction = model(data)
            loss = criterion(prediction * mask, gt * mask)
            total_loss += loss.item()
            total_ic += information_coefficient(gt * mask, prediction * mask).item()
            total_ric += rank_information_coefficient(gt * mask, prediction * mask).item()
            total_prec_10 += precision_at_k(gt * mask, prediction * mask, k=10).item()
            total_sharpe5 += sharpe_ratio(gt * mask, prediction * mask).item()
            count += 1
    avg_loss = total_loss / count
    avg_ic = total_ic / count
    avg_ric = total_ric / count
    avg_prec_10 = total_prec_10 / count
    avg_sharpe5 = total_sharpe5 / count
    return {
        'mse': avg_loss,
        'IC': avg_ic,
        'RIC': avg_ric,
        'prec_10': avg_prec_10,
        'sharpe5': avg_sharpe5
    }

# Training loop
best_valid_loss = float('inf')
best_valid_perf = None
best_test_perf = None

for epoch in range(epochs):
    model.train()
    np.random.shuffle(batch_offsets)
    train_loss = 0.0
    for j in range(valid_index - lookback_length - steps):
        data, mask, price, gt = get_batch(batch_offsets[j])
        optimizer.zero_grad()
        prediction = model(data)
        loss = criterion(prediction * mask, gt * mask)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= (valid_index - lookback_length - steps)
    print(f'Epoch {epoch + 1}, Training Loss: {train_loss:.4f}')

    val_perf = validate(valid_index, test_index)
    print('Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
        val_perf['mse'], val_perf['IC'], val_perf['RIC'], val_perf['prec_10'], val_perf['sharpe5']))

    test_perf = validate(test_index, len(mask_data))
    print('Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
        test_perf['mse'], test_perf['IC'], test_perf['RIC'], test_perf['prec_10'], test_perf['sharpe5']), '\n\n')

    if val_perf['mse'] < best_valid_loss:
        best_valid_loss = val_perf['mse']
        best_valid_perf = val_perf
        best_test_perf = test_perf

print('Best Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
    best_valid_perf['mse'], best_valid_perf['IC'], best_valid_perf['RIC'], best_valid_perf['prec_10'], best_valid_perf['sharpe5']))
print('Best Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
    best_test_perf['mse'], best_test_perf['IC'], best_test_perf['RIC'], best_test_perf['prec_10'], best_test_perf['sharpe5']))


Epoch 1, Training Loss: 0.0041
Valid performance:
 mse:6.69e-04, IC:-6.34e-03, RIC:3.93e-04, prec@10:6.58e-04, SR:-6.45e-01
Test performance:
 mse:5.70e-04, IC:-4.54e-03, RIC:5.10e-03, prec@10:3.06e-04, SR:-6.88e-01 


Epoch 2, Training Loss: 0.0005
Valid performance:
 mse:5.81e-04, IC:-3.18e-03, RIC:-2.13e-03, prec@10:7.88e-04, SR:-4.66e-01
Test performance:
 mse:4.80e-04, IC:-2.37e-03, RIC:-5.96e-03, prec@10:1.57e-04, SR:-4.92e-01 


Epoch 3, Training Loss: 0.0005
Valid performance:
 mse:5.06e-04, IC:5.23e-03, RIC:4.61e-03, prec@10:1.21e-03, SR:-2.00e-01
Test performance:
 mse:4.05e-04, IC:3.78e-03, RIC:-1.26e-02, prec@10:2.80e-04, SR:-2.00e-01 


Epoch 4, Training Loss: 0.0006
Valid performance:
 mse:5.99e-04, IC:3.07e-02, RIC:-7.22e-04, prec@10:1.96e-03, SR:5.74e-01
Test performance:
 mse:5.58e-04, IC:3.00e-02, RIC:-2.01e-02, prec@10:4.18e-03, SR:6.76e-01 


Epoch 5, Training Loss: 0.0006
Valid performance:
 mse:5.44e-04, IC:3.07e-02, RIC:-2.20e-03, prec@10:1.90e-03, SR:4.38e-01
Te